In [1]:
import concurrent.futures as cf
import os
import sys
sys.path.insert(0, os.path.abspath('..'))

from autocnet.examples import get_path
from autocnet.graph.network import CandidateGraph
from autocnet.graph.edge import Edge
from autocnet.matcher.matcher import FlannMatcher

from IPython.display import display

%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Generate a 3 image adjacenecy graph

Here we generate a three image adjacenecy graph, compute interest points, find matches, identify outliers, and finally compute fundamental matrices between all image pairs.  

The below example is threaded.

In [10]:
#Point to the adjacency Graph
adjacency = get_path('three_image_adjacency.json')
basepath = get_path('Apollo15')
cg = CandidateGraph.from_adjacency(adjacency, basepath=basepath)

#Apply SIFT to extract features
cg.extract_features(method='sift', extractor_parameters={'nfeatures':500})

#Match
with cf.ThreadPoolExecutor(max_workers=3) as ex:
    #Match
    jobs = [ex.submit(e.match) for s, d, e in cg.edges_iter(data=True)]
    [j.result() for j in jobs]
    
    # Symmetry Outlier
    jobs = [ex.submit(e.symmetry_check) for s, d, e in cg.edges_iter(data=True)]
    [j.result() for j in jobs]

    #Ratio Outlier
    jobs = [ex.submit(e.ratio_check) for s, d, e in cg.edges_iter(data=True)]
    [j.result() for j in jobs]

    #Compute the Fundamental Matrix (F)
    jobs = [ex.submit(e.compute_fundamental_matrix,
                      clean_keys=['symmetry', 'ratio']) for s, d, e in cg.edges_iter(data=True)]
    [j.result() for j in jobs]

2000 93 93
2000 94 94
2000 79 79


### Serial
As above, but this example utilizes a single core.  The API is potentially less complex to utilize, but the cost is overall performance.  These two examples, with just three small iamges, are too small to perform meaningfully different.

In [ ]:
#Point to the adjacency Graph
adjacency = get_path('three_image_adjacency.json')
basepath = get_path('Apollo15')
cg = CandidateGraph.from_adjacency(adjacency, basepath=basepath)

#Apply SIFT to extract features
cg.extract_features(method='sift', extractor_parameters={'nfeatures':500})

#Match
cg.match_features()

#Symmetry Outlier
cg.symmetry_checks()

#Ratio Outlier
cg.ratio_checks()

#Compute the Fundamental Matrix (F)
cg.compute_fundamental_matrices(clean_keys=['ratio', 'symmetry'], method='linear')

In [ ]:
figsize(20,20)
cg[1][2].plot(clean_keys=['fundamental'])

### The graph object:
The underlying data structure is a graph, where each node is an image and each edge is the connectivity between nodes.  Nodes and Edges are classes with associated attributes and methods.  This notebook primarily focuses on the plotting functionality on the graph (and graph components).

In these notebooks, the graph object is being stored in the variable `cg`.  Access to nodes and edges is positional.

  * To access a node in the graph:  `cg[node_idx]`, e.g. `cg[0]`.
  * To access an edge in the graph: `cg[source_idx][destination_idx]`, e.g. `cg[0][1]`
  

## Plot the graph

In [ ]:
cg.plot()

## Plot features at an individual node, e.g. a single image

All defaults are used here.

In [ ]:
cg.node[1].plot()

This example specifies a plotting layout, passing in an axis object and passes along a color.  All the MatPlotLib plotting arguments are supported.

In [ ]:
ax1 = plt.subplot(1,1,1)
ax = cg.node[0].plot(ax=ax1, color='y')

## Plotting Matches on an Edge
The plotting capability on a given node is limited to a single image; one can envision the node as being the image with all associated metadata and derived information.  The edge represents the overlap between images and resultant shared information, e.g. point correspondences, a homography, etc.

#### Plot the matches between an edge using two outlier detector masks
To get a rough idea of what a 'good' results should be, we should see no, or few, lines which intersect.

In [ ]:
fig, ax = plt.subplots(1,1)
ax = cg.edge[0][1].plot(clean_keys=['ratio', 'symmetry', 'fundamental'], ax=ax)

#### Now plot with the added, ransac computed mask

In [ ]:
cg.edge[0][1].plot(clean_keys=['ratio', 'symmetry', 'fundamental'], line_kwargs={'linewidth':0})

## Compute Coverage Metric
We compute a coverage metric by utilizing the homography to project the destination image corner pixel coordinates into the source image and computing the intersection.  This is a rough estimate that is as good (or poor) as the homography.

In [ ]:
H = cg.edge[0][1].homography
print('Not zero is good:', H.determinant)
print('Not huge is good: ', H.condition)
print('Shifts less than one pixel in all directions are good:', H.rmse)

In [ ]:
#Ideal coverage would be 1.0
cg.edge[0][1].coverage_ratio(clean_keys=['ransac'])

The above suggests that the quality is a function of the homography.  Just how good is the homography?  We can use the determinant (something near 1 is bad), the condition (a very large number, e.g. $10^15$ is bad), or the RMSE (reported in the x and y directions).

## Viewing Keypoint Information
Here we want to explore the attributes of the keypoints, using the masking information, e.g. the outlier detection methods.  The question is, what are the characteristics of those keypoints that have made it through the outlier detection.

In [ ]:
skp, dkp = cg.edge[0][1].keypoints(clean_keys=['ratio', 'symmetry', 'ransac'])
display(skp)
display(dkp)

## Subpixel Register
We suggest only subpixel registering 'good' candidate matches as the subpixel registration process can be time consuming.

In [ ]:
cg.edge[0][1].subpixel_register(clean_keys=['fundamental', 'symmetry', 'ratio'],template_size=5, search_size=15)

In [ ]:
cg.edge[0][1].plot(clean_keys=['ratio', 'symmetry', 'fundamental', 'subpixel'])

In [ ]:
cg.edge[0][1].plot(clean_keys=['ratio', 'symmetry', 'fundamental', 'subpixel'], line_kwargs={'linewidth':0})

## Apply Suppression vis Disk Covering
This method seeks to keep the $k$ strongest correlations with a good spatial distribution.

In [ ]:
cg.edge[0][1].suppress(k=100)

# Plot, in blue the points that passed all outlier detectors so far
cg.edge[0][1].plot(clean_keys=['ratio', 'symmetry', 'fundamental', 'subpixel'], line_kwargs={'linewidth':0})
# Overlay, in red, the points that remain after suppression
cg.edge[0][1].plot(clean_keys=['suppression'], line_kwargs={'linewidth':0}, scatter_kwargs={'color':'red'})